# **DEPENDENCIAS Y LIBRERÍAS**

In [ ]:
# Limpieza de ficheros
import os # Operaciones en ficheros
for f in os.listdir():  # buscar archivos de tipo Expr, .pý, .tokens
    if f.startswith("Expr") or (f.endswith(".py") or f.endswith(".tokens")):
        os.remove(f)  # limpia los encontrados

In [ ]:
# Instalar componente ANTLR4 para Python
!pip install antlr4-python3-runtime==4.13.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-python3-runtime-4.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
omegaconf 2.3.0 requires antlr4-python3-runtime==4.9.*, but you have antlr4-python3-runtime 4.13.1 which is incompatible.


In [ ]:
# Descargar el compilador de ANTLR4
!curl -O https://www.antlr.org/download/antlr-4.13.1-complete.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2089k  100 2089k    0     0  4274k      0 --:--:-- --:--:-- --:--:-- 4280k


# **CREAR GRAMÁTICA**

In [ ]:
# Reemplaza la celda donde defines 'gramatica' y escribes 'Expr.g4' con esto:

# Reglas de nuestra gramática

# Proximo objetivo:
# datosPacienteSentence diagnosticoSentence medicamentoSentence+ fechasSentence EOF

# Nombre gramatica: Expr

gramatica = """
grammar Expr;

// --- Reglas Sintácticas - Parser ---
prog:
    (registro)+ EOF
    ;

registro:
    datosPacienteSentence diagnosticoSentence medicamentoSentence+ fechasSentence
    ;

// --- SubReglas Sintácticas - Parser ---
datosPacienteSentence:
    'Paciente:' WS* PALABRAS ',' WS* INT INT INT INT INT INT INT INT
    ;

diagnosticoSentence:
    'Diagnostico:' WS* PALABRAS
    ;

fechasSentence:
    'Expede:' WS* FECHA WS* 'Caduce:' WS* FECHA
    ;

medicamentoSentence:
    'Medicamento:' WS* PALABRAS ',' WS* CONCENTRACION ',' WS* PALABRAS ',' WS* PALABRAS ',' WS* INT+ ',' WS* PALABRAS '.'
    ;

// --- Reglas Léxicas (Tokens) - Lexer ---
PALABRAS :   [a-zA-ZáéíóúÁÉÍÓÚñÑ]+ ( ' ' [a-zA-ZáéíóúÁÉÍÓÚñÑ]+ )* ;
INT :      [0-9] ;
CONCENTRACION : INT+ ' ' PALABRAS ;
FECHA :    INT INT '-' INT INT '-' INT INT ;

WS  : [ \\t\\r\\n]+ -> skip ;
"""

with open("Expr.g4", "w", encoding='utf-8', newline='\n') as f:
    f.write(gramatica)

# **GENERAR ARCHIVOS**

In [ ]:
# Genera varios archivos Lexer.py, Parser.py, .tokens, etc.
#   Sirven para interpretar y ejecutar texto segun las reglas de la gramatica
!java -jar antlr-4.13.1-complete.jar -Dlanguage=Python3 Expr.g4


# **PRUEBA DE GRAMÁTICA**

In [ ]:
from antlr4 import *
from ExprLexer import ExprLexer   # Importar Lexer del compilador
from ExprParser import ExprParser # Importar Parser del compilador

def parse_expression(input_text): # ingresa un String - texto
    input_stream = InputStream(input_text)  # analizar texto
    lexer = ExprLexer(input_stream)         # pasar al lexer
    token_stream = CommonTokenStream(lexer) # pasar a los tokens
    parser = ExprParser(token_stream)       # pasar al parser
    tree = parser.prog()                    # obtener arbol semantico
    return tree.toStringTree(recog=parser)  # convertir arbol a String

In [ ]:
# Prueba 1:
print(parse_expression("""Paciente: Juan Pérez Gómez, 12345678
                          Diagnostico: Gripe común
                          Medicamento: Paracetamol, 500 mg, Tableta, Oral, 5, Tomar cada ocho horas.
                          Medicamento: Ibuprofeno, 10 mg, Tableta, Oral, 5, Tomar cada cinco horas.
                          Expede: 01-05-25 Caduce: 01-06-25"""))

(prog (registro (datosPacienteSentence Paciente: Juan Pérez Gómez , 1 2 3 4 5 6 7 8) (diagnosticoSentence Diagnostico: Gripe común) (medicamentoSentence Medicamento: Paracetamol , 500 mg , Tableta , Oral , 5 , Tomar cada ocho horas .) (medicamentoSentence Medicamento: Ibuprofeno , 10 mg , Tableta , Oral , 5 , Tomar cada cinco horas .) (fechasSentence Expede: 01-05-25 Caduce: 01-06-25)) <EOF>)


In [ ]:
# Prueba 2:
print(parse_expression("""Paciente: Mario Chavez Raul, 12347478
                        Diagnostico: Tos fuerte
                        Medicamento: Paracetamol, 500 mg, Tableta, Oral, 5, Tomar cada ocho horas.
                        Expede: 02-05-25 Caduce: 01-10-25
                        Paciente: Carlos Méndez Ruiz, 11223344
                        Diagnostico: Sinusitis crónica
                        Medicamento: Cefalexina, 500 mg, Tableta, Oral, 14, Tomar cada ocho horas.
                        Medicamento: Loratadina, 10 mg, Tableta, Oral, 7, Tomar una vez al día.
                        Medicamento: Salbutamol, 100 mcg, Inhalador, Inhalatoria, 1, Usar cada seis horas si hay dificultad respiratoria.
                        Expede: 05-06-25 Caduce: 05-08-25"""))

(prog (registro (datosPacienteSentence Paciente: Mario Chavez Raul , 1 2 3 4 7 4 7 8) (diagnosticoSentence Diagnostico: Tos fuerte) (medicamentoSentence Medicamento: Paracetamol , 500 mg , Tableta , Oral , 5 , Tomar cada ocho horas .) (fechasSentence Expede: 02-05-25 Caduce: 01-10-25)) (registro (datosPacienteSentence Paciente: Carlos Méndez Ruiz , 1 1 2 2 3 3 4 4) (diagnosticoSentence Diagnostico: Sinusitis crónica) (medicamentoSentence Medicamento: Cefalexina , 500 mg , Tableta , Oral , 1 4 , Tomar cada ocho horas .) (medicamentoSentence Medicamento: Loratadina , 10 mg , Tableta , Oral , 7 , Tomar una vez al día .) (medicamentoSentence Medicamento: Salbutamol , 100 mcg , Inhalador , Inhalatoria , 1 , Usar cada seis horas si hay dificultad respiratoria .) (fechasSentence Expede: 05-06-25 Caduce: 05-08-25)) <EOF>)


In [ ]:
# Prueba 3
print(parse_expression("""Paciente: Juan Pérez Gómez, 12345678
                          Medicamento: Paracetamol, 500 mg, Tableta, Oral, 5, Tomar cada ocho horas.
                          Diagnostico: Gripe común
                          Medicamento: Ibuprofeno, 10 mg, Tableta, Oral, 5, Tomar cada cinco horas.
                          Expede: 01-05-25 Caduce: 01-06-25"""))

(prog (registro (datosPacienteSentence Paciente: Juan Pérez Gómez , 1 2 3 4 5 6 7 8) diagnosticoSentence (medicamentoSentence Medicamento: Paracetamol , 500 mg , Tableta , Oral , 5 , Tomar cada ocho horas .) Diagnostico: Gripe común (medicamentoSentence Medicamento: Ibuprofeno , 10 mg , Tableta , Oral , 5 , Tomar cada cinco horas .) (fechasSentence Expede: 01-05-25 Caduce: 01-06-25)) <EOF>)


line 2:26 mismatched input 'Medicamento:' expecting 'Diagnostico:'
line 3:26 extraneous input 'Diagnostico:' expecting {'Expede:', 'Medicamento:'}


In [ ]:
# Prueba 3
print(parse_expression("""Paciente: Luis Alberto Gómez, 556688
                          Diagnostico: Infección urinaria
                          Medicamento: Fenazopiridina, 100 mg, Tableta, Oral, p, Tomar cada ocho horas por tres días.
                          Expede: 15-06-25 Caduce: 15-07-25"""))

(prog (registro (datosPacienteSentence Paciente: Luis Alberto Gómez , 5 5 6 6 8 8) (diagnosticoSentence Diagnostico: Infección urinaria) (medicamentoSentence Medicamento: Fenazopiridina , 100 mg , Tableta , Oral , p , Tomar cada ocho horas por tres días .) (fechasSentence Expede: 15-06-25 Caduce: 15-07-25)) <EOF>)


line 2:26 mismatched input 'Diagnostico:' expecting INT
line 3:78 mismatched input 'p' expecting {INT, WS}
